In [1]:
!pip install -q langgraph langsmith langchain langchain_groq langchain_community langchain-core langchainhub langgraph-supervisor

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.2/151.2 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 65.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.7/437.7 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 11.1 MB/s eta 0:00:00


## Get API Key

In [2]:
import os
import getpass
if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API key: ")


Enter your Groq API key:  ········


## LLM Defined

In [3]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    groq_api_key = os.environ['GROQ_API_KEY'],
    model = 'gemma2-9b-it',
    temperature = 0.2
    
)
print(llm)

client=<groq.resources.chat.completions.Completions object at 0x79efd67f4e50> async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x79efd67ce210> model_name='gemma2-9b-it' temperature=0.2 model_kwargs={} groq_api_key=SecretStr('**********')


## State

In [11]:
from typing_extensions import TypedDict
from typing import Annotated
from langgraph.graph.message import add_messages

class State(TypedDict):
    message: Annotated[list, add_messages]

## Custom tools

In [12]:
from langgraph.prebuilt import create_react_agent
from langgraph_supervisor import create_supervisor

from langchain.tools import Tool

In [5]:
## define tool function 

def multiply(a: int, b: int) -> int:
    """Multiply a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b

mul_tool = Tool.from_function(
    name = 'Multiply',
    func = multiply,
    description = 'Multiplication of Two numbers'
)



def add(a: int, b: int) -> int:
    """
    Add a and b

    Args: 
    a: First int
    b: Second Integer
    """
    return a+b
    
add_tool = Tool.from_function(
    name = 'Add',
    func = multiply,
    description = 'Addition of Two numbers'
    
)




def divide(a: int, b: int) -> float:
    """
    Divide a and b

    Args: 
    a: First int
    b: Second Integer
    """
    return a/b
    
div_tool = Tool.from_function(
    name = 'Divide',
    func = multiply,
    description = 'Division of Two numbers'
    
)

In [13]:
Tool = [mul_tool, add_tool, div_tool ]


llm_with_tools = llm.bind_tools(tools = Tool)
print(llm_with_tools)

bound=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x79efd67f4e50>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x79efd67ce210>, model_name='gemma2-9b-it', temperature=0.2, model_kwargs={}, groq_api_key=SecretStr('**********')) kwargs={'tools': [{'type': 'function', 'function': {'name': 'Multiply', 'description': 'Multiplication of Two numbers', 'parameters': {'properties': {'__arg1': {'title': '__arg1', 'type': 'string'}}, 'required': ['__arg1'], 'type': 'object'}}}, {'type': 'function', 'function': {'name': 'Add', 'description': 'Addition of Two numbers', 'parameters': {'properties': {'__arg1': {'title': '__arg1', 'type': 'string'}}, 'required': ['__arg1'], 'type': 'object'}}}, {'type': 'function', 'function': {'name': 'Divide', 'description': 'Division of Two numbers', 'parameters': {'properties': {'__arg1': {'title': '__arg1', 'type': 'string'}}, 'required': ['__arg1'], 'type': 'object'}}}]} config={} config_factories=[]


In [14]:
## defining Chat Bot Node and Tool Node

def chat_bot(state: State)-> State:
    response = { 'message' : llm_with_tools.invoke(State['message'])}
    return response

tool_node = ToolNode(tools = Tool)

In [ ]:

builder = StateGraph(State)

# Define nodes
builder.add_node('Chat Bot', chat_bot)
builder.add_node('Tool ', tool_node)

## defining edges to connect


In [5]:
from langgraph.prebuilt import ToolNode, tools_condition  